<a href="https://colab.research.google.com/github/2020-nlp-c/nlp-statisticsmodel/blob/master/Sojeong_Cho/TextRank_%EB%AC%B8%EC%84%9C%EC%9A%94%EC%95%BD_%EC%A1%B0%EC%86%8C%EC%A0%95.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

TextRank: 문서요약

In [ ]:
nodes = "딸기 바나나 사과 파인애플 수박. 바나나 사과 딸기 포도. 복숭아 수박. 파인애플 사과 딸기 바나나."

In [ ]:
import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize, word_tokenize
import numpy as np
import math
import networkx as nx

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
# 토큰화
tokens = sent_tokenize(nodes)
tokens_t = np.array(tokens)
print(tokens_t)

token_1 = tokens_t[0].replace(".","")
print(token_1)
token_2 = tokens_t[1].replace(".","")
print(token_2)
token_3 = tokens_t[2].replace(".","")
print(token_3)
token_4 = tokens_t[3].replace(".","")
print(token_4)

['딸기 바나나 사과 파인애플 수박.' '바나나 사과 딸기 포도.' '복숭아 수박.' '파인애플 사과 딸기 바나나.']
딸기 바나나 사과 파인애플 수박
바나나 사과 딸기 포도
복숭아 수박
파인애플 사과 딸기 바나나


In [ ]:
token_s1 = token_1.split()
print(token_s1)
token_s2 = token_2.split()
print(token_s2)
token_s3 = token_3.split()
print(token_s3)
token_s4 = token_4.split()
print(token_s4)

doc = []

doc.append(token_s1)
doc.append(token_s2)
doc.append(token_s3)
doc.append(token_s4)

doc1 = np.array(doc)
doc1

['딸기', '바나나', '사과', '파인애플', '수박']
['바나나', '사과', '딸기', '포도']
['복숭아', '수박']
['파인애플', '사과', '딸기', '바나나']


array([list(['딸기', '바나나', '사과', '파인애플', '수박']),
       list(['바나나', '사과', '딸기', '포도']), list(['복숭아', '수박']),
       list(['파인애플', '사과', '딸기', '바나나'])], dtype=object)

In [ ]:
# 문장간 유사도

def Jaccard(t, d):
    union = set(t).union(set(d))
    intersection = set(t).intersection(set(d))
    return len(intersection)/len(union)

S = {}  
for i in range(len(doc1)):
    for j in range(len(doc1)):
        S.setdefault((i,j),Jaccard(doc[i], doc[j]))
S

{(0, 0): 1.0,
 (0, 1): 0.5,
 (0, 2): 0.16666666666666666,
 (0, 3): 0.8,
 (1, 0): 0.5,
 (1, 1): 1.0,
 (1, 2): 0.0,
 (1, 3): 0.6,
 (2, 0): 0.16666666666666666,
 (2, 1): 0.0,
 (2, 2): 1.0,
 (2, 3): 0.0,
 (3, 0): 0.8,
 (3, 1): 0.6,
 (3, 2): 0.0,
 (3, 3): 1.0}

In [ ]:
G_1 = np.zeros((4,4))
print(G_1)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [ ]:
# 그래프 엣지

for i in S:
    G_1[i] = S[i]
for i in range(len(G_1)):
    G_1[(i,i)] = 0

In [ ]:
print(G_1)

[[0.         0.5        0.16666667 0.8       ]
 [0.5        0.         0.         0.6       ]
 [0.16666667 0.         0.         0.        ]
 [0.8        0.6        0.         0.        ]]


In [ ]:
G_2 = np.array(G_1)
G_2

array([[0.        , 0.5       , 0.16666667, 0.8       ],
       [0.5       , 0.        , 0.        , 0.6       ],
       [0.16666667, 0.        , 0.        , 0.        ],
       [0.8       , 0.6       , 0.        , 0.        ]])

In [ ]:
# 엣지 가중치

E_W = []

for i in G_2:
    E_W.append(list(i/sum(i)))

print(E_W)

[[0.0, 0.3409090909090909, 0.11363636363636362, 0.5454545454545454], [0.45454545454545453, 0.0, 0.0, 0.5454545454545454], [1.0, 0.0, 0.0, 0.0], [0.5714285714285715, 0.4285714285714286, 0.0, 0.0]]


In [ ]:
# 최초 스코어

score = [0 for x in range(len(G_2))]

for i in range(len(G_2)) :
    temp_score = 0
    for j in range(len(G_2)):
        temp_score += G_2[j][i]
    score[i] = temp_score
score

[1.4666666666666668, 1.1, 0.16666666666666666, 1.4]

In [ ]:
d = 0.85
prev_score = score
for i in range(len(score)) :
    score[i] = (1 - d) + d*(score[i]) 
score

[1.396666666666667, 1.085, 0.2916666666666667, 1.3399999999999999]

In [ ]:
# 스코어 반복

score_cp = 0

while True :
    for i in range(len(G_2)) :
        tmp_score = 0
        for j in range(len(G_2)):
            tmp_score += G_2[j][i]
        score[i] = tmp_score
    print('이전 스코어 : ', score)

    d = 0.85
    p_score = score
    for i in range(len(score)) :
        score[i] = (1 - d) + d*(score[i]) 
    print('현재 스코어 : ', score)

    print("===================================================")

    for i in range(len(E_W)) :
        for j in range(len(E_W)) :
                G_2[i][j] = score[i]*E_W[i][j]
    print(G_2)

    print("===================================================")

    if  abs(score_cp - score[0]) < 0.0001 :
        break
    else :
        score_cp = score[0]

이전 스코어 :  [1.4829914956369616, 1.0290860112465106, 0.1602933975048963, 1.3283641921192624]
현재 스코어 :  [1.4105427712914174, 1.024723109559534, 0.2862493878791619, 1.2791095633013732]
[[0.         0.48086685 0.16028895 0.76938697]
 [0.46578323 0.         0.         0.55893988]
 [0.28624939 0.         0.         0.        ]
 [0.73091975 0.54818981 0.         0.        ]]
이전 스코어 :  [1.4829523699550595, 1.0290566666927925, 0.16028895128311557, 1.328326844100519]
현재 스코어 :  [1.4105095144618005, 1.0246981666888737, 0.2862456085906483, 1.2790778174854411]
[[0.         0.48085552 0.16028517 0.76936883]
 [0.46577189 0.         0.         0.55892627]
 [0.28624561 0.         0.         0.        ]
 [0.73090161 0.54817621 0.         0.        ]]
